### Milestone 4: Deep learning, due Wednesday, April 26, 2017

For this milestone you will (finally) use deep learning to predict movie genres. You will train one small network from scratch on the posters only, and compare this one to a pre-trained network that you finetune. [Here](https://keras.io/getting-started/faq/#how-can-i-use-pre-trained-models-in-keras) is a description of how to use pretrained models in Keras.

You can try different architectures, initializations, parameter settings, optimization methods, etc. Be adventurous and explore deep learning! It can be fun to combine the features learned by the deep learning model with a SVM, or incorporate meta data into your deep learning model. 

**Note:** Be mindful of the longer training times for deep models. Not only for training time, but also for the parameter tuning efforts. You need time to develop a feel for the different parameters and which settings work, which normalization you want to use, which model architecture you choose, etc. 

It is great that we have GPU's via AWS to speed up the actual computation time, but you need to be mindful of your AWS credits. The GPU instances are not cheap and can accumulate costs rather quickly. Think about your model first and do some quick dry runs with a larger learning rate or large batch size on your local machine. 

The notebook to submit this week should at least include:

- Complete description of the deep network you trained from scratch, including parameter settings, performance, features learned, etc. 
- Complete description of the pre-trained network that you fine tuned, including parameter settings, performance, features learned, etc. 
- Discussion of the results, how much improvement you gained with fine tuning, etc. 
- Discussion of at least one additional exploratory idea you pursued

# Milestone 4 submission
## Team 37 (Alexander Dubitskiy, Keenan Venuti, Timur Zambalayev)

Github repo link: https://github.com/adubitskiy/cs109b

##  Complete description of the deep network you trained from scratch, including parameter settings, performance, features learned, etc. 

Here is the link for the notebook: https://github.com/adubitskiy/cs109b/blob/master/model_custom_loss.ipynb

The data consisted of 58,825 movies. We reserved 10% for the test dataset (5,883 samples). The train set had 52,942 movies. 
During the fit we set aside 20% for validation (10,589 samples), so we trained on 42,353 samples.
The image sizes: 224x224x3 (rgb).

We used 4 convolution layers and 2 fully conected layers regularized by the dropouts.
The loss function was weighted crossentropy. Batch size was 128.

First we did rough tuning using Nesterov Adam optimizer (optimizer = 'nadam'). We used early stopping (stop when vaidation loss is not going down for 5 epochs). The rough tuning lasted for 17 epochs (each epoch taking around 200 seconds).
The train loss went from 29.2 to 8.9. The validation loss - from 26.9 to 21.6.

After that it was more precise tuning with a small step size (optimizer=SGD(lr = 0.0001, momentum=0.9)).
That took 7 additional epochs.

## Complete description of the pre-trained network that you fine tuned, including parameter settings, performance, features learned, etc. 

## Discussion of the results, how much improvement you gained with fine tuning, etc. 

##  Discussion of at least one additional exploratory idea you pursued